Connect to the Google drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Import libraries we use

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras import models
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from PIL import Image

Define image dimensions, batch size and epochs

In [4]:
image_height = 256
image_width = 256
batch_size = 128
epochs = 10

Define paths to your dataset

In [5]:
train_directory = '/gdrive/MyDrive/Machine Learning/ADdatasets/train/foods_train_pro'
test_directory = '/gdrive/MyDrive/Machine Learning/ADdatasets/test/foods_test_pro'
validation_directory = '/gdrive/MyDrive/Machine Learning/ADdatasets/validation/foods_validation_pro'

Define the CNN model and compile it

In [6]:
'''
Create image classification model
'''
def create_model():
    model = Sequential()

    # create a CNN model
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(image_height, image_width, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=512, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units=512, activation='relu'))
    model.add(Dropout(0.5))

    # output layer: performs classification
    # 101 possible classes
    model.add(Dense(units=19, activation='softmax'))

    # build the model
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    return model

Main program

In [13]:
'''
Main program.
'''
def main():
    # create our CNN model
    model = create_model()

    # model architecture
    print(model.summary())

    '''
        Here, we use a data generator to feed in our data to our model.
        This is useful when we have a large dataset and we do not want
        to load all of it into memory at once. Instead, we can load a
        batch of data at a time.
    '''
    # RGB values are from 0 to 255; rescale each value by 1/255
    ImageFlow = ImageDataGenerator(
        rescale=1/255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # specify which directory our train data should be read from
    train_generator = ImageFlow.flow_from_directory(
        directory=train_directory,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb',
        subset='training'
    )

    # specify which directory our validation data should be read from
    validation_generator = ImageFlow.flow_from_directory(
        directory=validation_directory,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='rgb',
        subset='validation'
    )

    # compute the number of batches per epoch
    # the symbol // means to divide and floor the result
    steps_per_epoch = train_generator.n//train_generator.batch_size

    # train our model by feeding in the data generator
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        batch_size=batch_size,
        epochs=100,
        validation_data=validation_generator
    )

    # test how well our model performs against data
    # that it has not seen before
    # model.evaluate(x=x_test/255, y=y_test)
    model.evaluate(validation_generator)

    #plt.plot(history.history['accuracy'], label='accuracy')
    #plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    #plt.xlabel('Epoch')
    #plt.ylabel('Accuracy')
    #plt.ylim([0, 1])
    #plt.legend(loc='lower right')
    #plt.show()


    #ax = plt.subplots()

    #plt.plot(history.history['loss'])
    #ax.set_xlabel('Epochs')
    #ax.set_ylabel('Error')
    #ax.set_title('Loss curve')

    #plt.show()

    def load_and_preprocess_image(img_path, target_size=(image_height, image_width)):
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0

        return img_array

    def test_model(model, test_directory, class_labels):
        test_images = [f for f in os.listdir(test_directory) if f.endswith('.jpg')]

        image_names = []
        actual_class = []
        predicted_class = []

        for image_name in test_images:
            img_path = os.path.join(test_directory, image_name)
            img_array = load_and_preprocess_image(img_path)

            predictions = model.predict(img_array)
            predicted_label = np.argmax(predictions)
            actual_label = image_name.split('_')[0]

            image_names.append(image_name)
            actual_class.append(actual_label)
            predicted_class.append(predicted_label)

        results_df = pd.DataFrame({'Image': image_names, 'Actual Class': actual_class, 'Predicted Class': predicted_class})
        results_df['Predicted Class'] = results_df['Predicted Class'].map(class_labels)

        return results_df

    def load_class_labels(filename):
        with open(filename, 'r') as file:
            class_labels = file.read().splitlines()
        return class_labels

    def save_results_and_display_accuracy(results_df, save_path):
        results_df.to_csv(save_path, index=False)

        correct_predictions = np.sum(np.array(results_df['Actual Class']) == np.array(results_df['Predicted Class']))
        total_predictions = len(results_df)
        accuracy = correct_predictions / total_predictions

        print(f'Test Accuracy: {accuracy * 100}%')

    # Load the trained model
    model_path = '/gdrive/MyDrive/Machine Learning/ADmodels/food_recognition_model.h5'
    model.save(model_path)
    trained_model = models.load_model(model_path)

    #
    class_labels = load_class_labels('/gdrive/MyDrive/Machine Learning/ADdatasets/txt/classes.txt')

    # Test the model
    results_df = test_model(trained_model, test_directory, class_labels)

    # Save results and display accuracy
    save_path = '/gdrive/MyDrive/Machine Learning/ADresults/result.csv'
    save_results_and_display_accuracy(results_df, save_path)

In [14]:
# running app
if __name__ == '__main__':
  main();

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 127, 127, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_21 (MaxPooli  (None, 62, 62, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_22 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_22 (MaxPooli  (None, 30, 30, 128)      

InvalidArgumentError: Graph execution error:

Detected at node categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <la